In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("titanic/train.csv")
train_data_copy = train_data.copy()
train_data_copy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("titanic/test.csv")
test_data_copy = test_data.copy()
test_data_copy.head()
test_Ids = test_data_copy['PassengerId']

In [4]:
test_data_copy.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
def clean(train_data_copy):
    train_data_copy = train_data_copy.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    
    # fill missing values
    cols = ['Age', 'SibSp', 'Parch', 'Fare']
    for col in cols:
        train_data_copy[col].fillna(train_data_copy[col].mean(), inplace=True)
    
    train_data_copy.Embarked.fillna('U', inplace=True)
    return train_data_copy

In [6]:
test_data_copy = clean(test_data_copy)
test_data_copy

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.50000,0,0,7.8292,Q
1,3,female,47.00000,1,0,7.0000,S
2,2,male,62.00000,0,0,9.6875,Q
3,3,male,27.00000,0,0,8.6625,S
4,3,female,22.00000,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,30.27259,0,0,8.0500,S
414,1,female,39.00000,0,0,108.9000,C
415,3,male,38.50000,0,0,7.2500,S
416,3,male,30.27259,0,0,8.0500,S


In [7]:
train_data_copy = clean(train_data_copy)
train_data_copy

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [8]:
# map sex and embarked to values
from sklearn import preprocessing
# labelled encoder
lbe = preprocessing.LabelEncoder()

cols = ['Sex', 'Embarked']

for col in cols:
    # fit and transform
    train_data_copy[col] = lbe.fit_transform(train_data_copy[col])
    # transform test set
    test_data_copy[col] = lbe.transform(test_data_copy[col])
    print(lbe.classes_)
    
train_data_copy.head()

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [9]:
# model -> logisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y_target = train_data_copy['Survived']
x_lab = train_data_copy.drop('Survived', axis=1)

x_train, x_val, y_train, y_val = train_test_split(x_lab, y_target, test_size=0.2, random_state=42)

In [10]:
# classifier
clf = LogisticRegression(random_state=0, max_iter=1000).fit(x_train, y_train)

In [11]:
predictions = clf.predict(x_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

In [12]:
sub_prediction = clf.predict(test_data_copy)

In [13]:
df = pd.DataFrame({"PassengerId": test_Ids.values,
                  "Survived": sub_prediction,
                  })

In [16]:
df.to_csv('mysubmission.csv', index=False)